# Introdução

Neste documento faremos o preprocessamento dos textos para criar os casos testes e salvá-los. Dessa forma, poderemos retestar sem aleatoriedade de eventos.

In [1]:
# !pip install langchain==0.0.340 langchain --force-reinstall

In [1]:
from datetime import date

In [2]:
validation_tests_partition = str(date.today())

In [3]:
validation_tests_path = f'./validation_tests/{validation_tests_partition}'
validation_tests_path

'./validation_tests/2024-05-11'

# 1. Recuperação dos Documentos Formatados

In [4]:
import os
import json

In [5]:
pdf_folder_path = "./formatted_documents"

In [6]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [7]:
len(filename_list)

1538

In [8]:
def retrieve_document(filename):
    with open(filename, "r") as file:
        json_file = json.load(file)
    return json_file

## Teste

In [9]:
json_document = retrieve_document(filename_list[0])

In [10]:
json_document.keys()

dict_keys(['title', 'authors', 'abstract', 'bibliography_pages', 'keywords', 'urls', 'pdf_url', 'id', 'sentences'])

In [11]:
json_document['title']

'Psicologia, trabalho e atenção básica do SUS : compartilhando experiências em equipes NASF no Rio de Janeiro'

In [12]:
json_document['authors']

['Ambrósio, Erika Gonçalves',
 'Ferreira, João Batista, orient.',
 'Universidade Federal do Rio de Janeiro. Instituto de Psicologia.']

In [13]:
json_document['abstract'][:300] + '...'

'Esta pesquisa analisa a organização do trabalho de psicólogos nas equipes do Núcleo de Apoio à Saúde da Família (NASF) do município do Rio de Janeiro (área programática 2.1). Para tanto, apresenta o contexto de trabalho atual dos psicólogos no âmbito da atenção básica, identifica o trabalho prescrit...'

In [14]:
json_document['keywords']

['Psicologia Teses.',
 'Psicólogos Condições de trabalho Rio de Janeiro (RJ).',
 'Política de saúde.']

In [15]:
json_document['urls']

['http://objdig.ufrj.br/30/teses/276189.pdf',
 'http://lattes.cnpq.br/4763936516818542']

In [16]:
print(len(json_document["sentences"]))

411


# 2. Seleção dos Testes

**Escolha das sentenças do texto:** 

Usaremos apenas uma fração, temos 1500 artigos, selecionaremos 100 sentences ao acaso por artigo, totalizando 150000 sentenças.<br>
Além disso, faremos alterações nas sentenças obtidas, para testar segmentos do parágrafo.<br>
As sentenças precisam ter pelo menos uma quantidade de caracteres.

In [15]:
MIN_ACCEPTABLE_CHARS = 300
MAX_SENTENCE_CHUNK_SIZE = 500

In [16]:
import random

random.seed(42)

In [17]:
def chunkstring(string, length):
    return [string[0+i:length+i] for i in range(0, len(string), length)]

In [18]:
test_title = []
test_abstract = []
test_keywords = []
test_sentences = []
test_modified_sentences = []

metadata_dict = {}

for filename in filename_list:
    doc = retrieve_document(filename).copy()
   
    test_title.append({"id": doc["id"], "title": doc["title"]})
    test_abstract.append({"id": doc["id"], "abstract": doc["abstract"]})
    test_keywords.append({"id": doc["id"], "keywords": "; ".join(doc["keywords"])})
        
    acceptable_sentences = [sentence for sentence in doc["sentences"] if len(sentence) > MIN_ACCEPTABLE_CHARS]
    
    sentences = random.sample(acceptable_sentences, len(acceptable_sentences))
    for sentence in sentences:
        chunked_sentence = [{"id": doc["id"], "sentence": s} for s in chunkstring(sentence, MAX_SENTENCE_CHUNK_SIZE)]
        test_sentences.extend(chunked_sentence)
        
        start = random.randint(0, int(len(sentence)/2))
        test_modified_sentences.append({"id": doc["id"], "sentence": sentence[start:start+200]})
    
    metadata = doc.copy()
    metadata.pop('sentences')
    metadata_dict[doc['id']] = metadata

In [19]:
test_title[:2]

[{'id': '276189',
  'title': 'Psicologia, trabalho e atenção básica do SUS : compartilhando experiências em equipes NASF no Rio de Janeiro'},
 {'id': '494080',
  'title': 'Terapeutas cognitivo-comportamentais frente à religiosidade/espiritualidade no contexto psicoterapêutico'}]

In [20]:
test_abstract[:1]

[{'id': '276189',
  'abstract': 'Esta pesquisa analisa a organização do trabalho de psicólogos nas equipes do Núcleo de Apoio à Saúde da Família (NASF) do município do Rio de Janeiro (área programática 2.1). Para tanto, apresenta o contexto de trabalho atual dos psicólogos no âmbito da atenção básica, identifica o trabalho prescrito e o realizado; as vivências de sofrimento dos psicólogos em seu contexto de trabalho; assim como as vivências de prazer e estratégias utilizadas para a efetivação da prática profissional. A metodologia caracteriza-se pela realização de entrevistas semiestruturadas com seis psicólogas e um psicólogo, todos de equipes NASF (eNASF) da área da Zona Sul do município. As entrevistas ocorreram entre fevereiro de 2019 e fevereiro de 2020, foram gravadas e transcritas, e posteriormente analisadas por meio da Análise dos Núcleos de Sentido. O contexto de trabalho presente no município do Rio caracteriza-se por: redução gradual de investimentos na Saúde; transferência

In [21]:
test_keywords[:2]

[{'id': '276189',
  'keywords': 'Psicologia Teses.; Psicólogos Condições de trabalho Rio de Janeiro (RJ).; Política de saúde.'},
 {'id': '494080',
  'keywords': 'Psicologia Teses.; Psicologia e religião.; Religiosidade.; Espiritualidade.; Terapia cognitiva.'}]

In [22]:
test_sentences[:2]

[{'id': '276189',
  'sentence': 'A partir de 1994, o PACS foi adotado como estratégia transitória para o Programa de Saúde da Família, que se comprometia com o acesso universal e integral ao SUS, numa perspectiva de integralidade do cuidado em saúde, mas que ainda aglutinava práticas focais em áreas com mais riscos (REIS e BORGES, 2016).'},
 {'id': '276189',
  'sentence': 'A partir de março de 2020, em virtude da pandemia da COVID -19 em destaque no país, num movimento que se iniciou na China e foi se ocidentalizando ao longo dos dias, temas como saúde e segurança do trabalho, organização e adoecimento dos profissionais de sa úde, e a relevância do SUS na sociedade brasileira dominaram a programação televisiva, as manchetes jornalísticas e mudaram a rotina de milhões de pessoas. Nesse sentido, Lima (2020) ressalta que “mais do que nunca, a saúde como valor e direi'}]

In [23]:
test_modified_sentences[:2]

[{'id': '276189',
  'sentence': 'e se comprometia com o acesso universal e integral ao SUS, numa perspectiva de integralidade do cuidado em saúde, mas que ainda aglutinava práticas focais em áreas com mais riscos (REIS e BORGES, 2016'},
 {'id': '276189',
  'sentence': 'aúde da Família e Comunidade do Instituto de Atenção à Saúde São Francisco de Assis da Universidade Federal do Rio de Janeiro (HESFA/UFRJ) em 2016, inicio minha trajetória profissional como psicóloga.'}]

# 3. Salvamento dos Testes

In [31]:
import json

In [40]:
import os
os.mkdir(validation_tests_path)

In [41]:
with open(f'{validation_tests_path}/test_title.json', 'w') as fp:
    json.dump(test_title, fp)

In [42]:
with open(f'{validation_tests_path}/test_abstract.json', 'w') as fp:
    json.dump(test_abstract, fp)

In [43]:
with open(f'{validation_tests_path}/test_keywords.json', 'w') as fp:
    json.dump(test_keywords, fp)

In [44]:
with open(f'{validation_tests_path}/test_sentences.json', 'w') as fp:
    json.dump(test_sentences, fp)

In [45]:
with open(f'{validation_tests_path}/test_modified_sentences.json', 'w') as fp:
    json.dump(test_modified_sentences, fp)